# Deep Learning
> A catalogue of not so deep ideas in Deep Learning.
- toc: true 
- badges: true
- comments: true
- categories: [Research]


Deep Learning is a collection of techniques for finding functions that can approximate arbitrary input-output mappings, while capturing enough structure of the problem to be able to use them in practical applicatons.language-translation systems,Image classification,Movie Recommendation systems are Some notable applications where they have become defacto-choice. This is by no means an exhaustive treatment of the field. Here I provide concise summaries,followed by a simple implementation of some of the most interesting ideas in the field. I believe that many things in the field are unnecessarily complicated by lengthy treatment where a readable code and short explanation will suffice. 

`Note` : This is not intended to be the first introduction to deep learning. Here I wanted to succintly catalogue some the latest advancements in the field. But, respecting tradition the first module starts from the basics.

## The Basics

Many deep learning models follow a simple recipe:

    1. Gather the data.
    2. Define learnable parameters. And specify how they will interact with the data.(architecture)
    3. Define a loss function to minimize.
    4. Adjust the parameters until satisfied.

### Train a linear regression model using gradient-descent.

Here we will see how we can perform all the above steps starting with the most barebones implementation. Note that the procedure outlined here is general purpose - meaning the way we adjust `parameters` is going to remain same irrecpective of the modality of the data, details of the loss function or the architecture.

1. Gather the data

Let's generate some fake data.Let's assume that the data is coming from $y = 2*x1 - 4.2*x2 + 1 + noise(measurement error)$. This can be more succintly represented in vector notation :
$$y = \begin{bmatrix} x1 \\ x2 \end{bmatrix} . \begin{bmatrix} 2 \\ -4.2 \end{bmatrix} + 1$$


    


In [7]:
import numpy as np
def get_data(params,const=None,rows=1000):
    #number of features in the input
    dim = len(params)
    x = np.random.normal(0,0.3,(rows,dim))
    y = x@params
    if const:
        y += const
    return x,y

In [9]:
x,y = get_data(np.array([2,-4.2]))

In [12]:
x.shape,y.shape

((1000, 2), (1000,))

Here we know the parameters of the model `[2,-4.2]`. Our task is to learn them by just using input-output mappings `(x,y)`. Ideally we even don't have to extract the orginal parameters but only be able to reduce some predefined loss. But for now, let's just see if we could extract the orginal parameters using the data. 

In [2]:
# TODO: From scratch - implement backprop in numpy

In [15]:
# From scratch using pytorch

def model():
    pass

In [ ]:
# More about pytorch

In [14]:
# using pytorch to building more flexible architecure.

Some Good ideas to know

DropOut : While training some of the hidden layers can be randomly put to zero. This forces our network to learn more robust representations. Dropout can also break symmetry.

Break Symmetry : Initialize weights randomly to break symmetry.

Regularization methods: Weight Decay.

Read [Convolutions Arthmetic](https://arxiv.org/pdf/1603.07285.pdf)